# Jupyter notebook 02: Analysis of Collaborative Toponyms in OpenStreetMap

***Paper: Collaborative Toponyms in OpenStreetMap: an open-source framework to investigate the relationship with intrinsic quality parameters***

**Aims**

- To conduct a quantitative assessment of elements within OpenStreetMap (OSM) that have the 'name' attribute filled for potential categories of the Brazilian Authoritative Topographic Map; and

- To investigate the most significant intrinsic quality parameters that contribute to the reliability of toponyms in OSM.


**Brief Overview of the Proposed Methodology**

- Preliminary survey of potential OpenStreetMap (OSM) tags to provide relevant toponym information to categories of interest related to Brazilian Topographic Mapping;

- Execution of a quantitative analysis on collaboratively entered toponyms, utilizing homogeneous grid-based approaches; and

- Assessment of intrinsic quality parameters as indicators of the reliability of toponyms in a scientific context.

---

## Install the necessary libraries to the analysis

In [ ]:
# Just in case of using colab, install these necessary libraries
%pip install geopandas matplotlib pysal seaborn mapclassify mgwr -q

## Import the libraries

In [ ]:
# Import library and some pre-installed modules
import os
import sys
import numpy as np
import json
import folium
import pandas as pd
import geopandas as gpd
import mapclassify
import matplotlib.colors
import seaborn as sns
import statsmodels.api as sm
import warnings
import pyproj
import branca.colormap as cm
from matplotlib import pyplot as plt
from ipywidgets import widgets, Layout, Button, interact, Dropdown, SelectMultiple, HBox, VBox, Output
from IPython.display import display, clear_output, FileLink
from folium import plugins, Map, Element, Figure, LayerControl, TileLayer
from folium.features import GeoJson, GeoJsonTooltip,Choropleth, CircleMarker
from pysal.explore import esda
from pysal.lib import weights
from shapely.geometry import MultiPolygon, box
from shapely.ops import transform
from jinja2 import Template
from tqdm.notebook import tqdm
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from esda import G_Local
from esda.moran import Moran
from splot.esda import plot_moran
from statsmodels.stats.outliers_influence import variance_inflation_factor


%matplotlib inline

In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')


In [ ]:
# Check the current working directory
os.getcwd()

In [4]:
# Check the current directories in the working directory
%ls

LICENSE     data/       notebooks/  utils/
README.md   deprecated/ results/


## Connect to Google Drive (*just in case of using Google Colaboratory environment*)

In [ ]:
# Cell to run just in case of using colab, conect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Analysis of Collaborative Toponyms in OpenStreetMap

### Import the regular grid cells with the aggregated data extracted from OSM via the OHSOME API
 * Results from the Algorithm 1 implemented in Jupyter Notebook 1 (JN1)

In [51]:
# Import the grid with the aggregated data extracted from OSM via the OHSOME API
grid_osm = None

# Output to display messages
output = widgets.Output()

# Base path for the directories with GeoJSON files
base_path = 'data/output_code1'

# Store the name of the selected GeoJSON file
selected_file = None

# Function for selecting and loading the GeoJSON file
def select_file(change):
    global grid_osm, selected_file
    selected_file = change['new']
    
    if selected_file != "Select the GeoJSON file of the case study area:":
        file_path = os.path.join(base_path, selected_file)
        try:
            with open(file_path, 'r') as file:
                grid_osm = json.load(file)
            with output:
                clear_output(wait=True)
                print("File selected: %s loaded with success!" %(selected_file))
                print("\nFile path:", file_path)
        except FileNotFoundError:
            with output:
                clear_output(wait=True)
                print("File not found:", selected_file)

# Listing available GeoJSON files
file_list = [f for f in os.listdir(base_path) if f.endswith('.geojson')]
options = ["Select the GeoJSON file of the case study area:"] + file_list

# Dropdown to select the GeoJSON file
dropdown = widgets.Dropdown(options=options)
dropdown.observe(select_file, names='value')

# Display the dropdown
display(dropdown, output)

Dropdown(options=('Select the GeoJSON file of the case study area:', 'grade_id77_passare_results.geojson', 'gr…

Output()

In [52]:
# Preview grid cells in GeoJSON format
display(grid_osm)

{'type': 'FeatureCollection',
 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4674'}},
 'features': [{'type': 'Feature',
   'properties': {'id': '200ME67074N108676',
    'POP10': 3,
    'leisure_total_count': 0.0,
    'leisure_name_count': 0.0,
    'leisure_name_ratio': 0,
    'building_total_count': 7.0,
    'building_name_count': 0.0,
    'building_name_ratio': 0.0,
    'amenity_total_count': 0.0,
    'amenity_name_count': 0.0,
    'amenity_name_ratio': 0,
    'leisure_total_contributions': 0.0,
    'leisure_name_contributions': 0.0,
    'building_total_contributions': 39.0,
    'building_name_contributions': 0.0,
    'amenity_total_contributions': 0.0,
    'amenity_name_contributions': 0.0,
    'leisure_latest5_name_contributions': 0.0,
    'building_latest5_name_contributions': 0.0,
    'amenity_latest5_name_contributions': 0.0,
    'leisure_name_tagChange_contributions': 0.0,
    'building_name_tagChange_contributions': 0.0,
    'amenity_name_tagChange_contr

### Convert GeoJSON Data to GeoDataFrame

In [53]:
# Convert GeoJSON Data to GeoDataFrame
gdf_osm = gpd.GeoDataFrame.from_features(grid_osm['features'])

In [54]:
# Check the five first and last records of GeoDataFrame
display(gdf_osm)

,geometry,id,POP10,leisure_total_count,leisure_name_count,leisure_name_ratio,building_total_count,building_name_count,building_name_ratio,amenity_total_count,...,amenity_name_contributions,leisure_latest5_name_contributions,building_latest5_name_contributions,amenity_latest5_name_contributions,leisure_name_tagChange_contributions,building_name_tagChange_contributions,amenity_name_tagChange_contributions,leisure_users_count_name,building_users_count_name,amenity_users_count_name
0,"MULTIPOLYGON (((-38.54451 -3.82662, -38.54461 ...",200ME67074N108676,3,0.0,0.0,0.0,7.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"MULTIPOLYGON (((-38.54271 -3.82652, -38.54281 ...",200ME67076N108676,166,2.0,0.0,0.0,29.0,0.0,0.000000,3.0,...,14.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,5.0
2,"MULTIPOLYGON (((-38.54090 -3.82643, -38.54100 ...",200ME67078N108676,562,0.0,0.0,0.0,33.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"MULTIPOLYGON (((-38.54461 -3.82483, -38.54471 ...",200ME67074N108678,223,0.0,0.0,0.0,78.0,0.0,0.000000,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,"MULTIPOLYGON (((-38.54281 -3.82473, -38.54291 ...",200ME67076N108678,62,0.0,0.0,0.0,49.0,0.0,0.000000,2.0,...,3.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,"MULTIPOLYGON (((-38.51811 -3.81255, -38.51821 ...",200ME67104N108690,373,0.0,0.0,0.0,155.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
216,"MULTIPOLYGON (((-38.51631 -3.81245, -38.51641 ...",200ME67106N108690,369,1.0,0.0,0.0,136.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,7.0,0.0,0.0
217,"MULTIPOLYGON (((-38.52183 -3.81095, -38.52193 ...",200ME67100N108692,219,1.0,1.0,100.0,98.0,0.0,0.000000,2.0,...,7.0,1.0,0.0,2.0,8.0,0.0,3.0,6.0,0.0,5.0
218,"MULTIPOLYGON (((-38.52002 -3.81085, -38.52012 ...",200ME67102N108692,289,0.0,0.0,0.0,122.0,3.0,2.459016,1.0,...,7.0,0.0,3.0,2.0,0.0,6.0,3.0,0.0,6.0,7.0


In [55]:
# Check the number os grid cell with information retrived of OSM
len(gdf_osm)

220

In [56]:
gdf_osm.crs

In [57]:
print("CRS is None?", gdf_osm.crs)

if gdf_osm.crs is None:
    gdf_osm = gdf_osm.set_crs(epsg=4674)

print("\nCRS defined:")
gdf_osm.crs

CRS is None? None

CRS defined:


<Geographic 2D CRS: EPSG:4674>
Name: SIRGAS 2000
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Latin America - Central America and South America - onshore and offshore. Brazil - onshore and offshore.
- bounds: (-122.19, -59.87, -25.28, 32.72)
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

---
### Quantitative Analysis

#### 1. Preliminary Exploratory analysis

##### 1.1 Histogram Analysis

In [ ]:
# Preliminary analysis with histograms

# Setup directory for saving histograms
histogram_output_dir = 'results/1_descriptive_statistics/1_histograms/'
os.makedirs(histogram_output_dir, exist_ok=True)

# Extract study area name function (based on selected_file)
def extract_study_area_name(selected_file):
    study_area_name, _ = os.path.splitext(selected_file)
    tokens = study_area_name.replace("-", "_").split("_")
    try:
        idx_results = tokens.index('results')
        study_area_token = tokens[idx_results - 1]
    except ValueError:
        study_area_token = tokens[-1]
    return study_area_token.replace("-", " ").replace("_", " ").title()

# Ensure that selected_file is already defined elsewhere
study_area_name_only = extract_study_area_name(selected_file)

# --- Widgets ---
dropdown = widgets.Dropdown(
    options=[col for col in gdf_osm.columns if col != 'geometry'],
    value='leisure_name_ratio',
    description='Column:',
    disabled=False
)

plot_button = widgets.Button(
    description='Generate Histogram',
    button_style='success'
)

save_button = widgets.Button(
    description='Save Histogram as PNG',
    button_style='info',
    disabled=True
)

output = widgets.Output()

# --- Functions ---
def plot_histogram(b=None):
    clear_output(wait=True)
    display(widgets.VBox([dropdown, plot_button, save_button, output]))

    column = dropdown.value
    with output:
        output.clear_output()
        plt.figure(figsize=(8, 5))
        sns.histplot(gdf_osm[column].dropna(), kde=True, bins=20)  # kde=True adds the density smoothed line
        plt.title(f'Histogram of {column} of {study_area_name_only}', fontsize=16)
        plt.xlabel(f'Grid cells values of {column}', fontsize=16)
        plt.ylabel('Frequency', fontsize=16)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.grid(False)
        plt.tight_layout()
        plt.show()

        save_button.disabled = False

def save_histogram(b):
    column = dropdown.value
    save_path = os.path.join(
        histogram_output_dir,
        f'histogram_{column.lower()}_{study_area_name_only.replace(" ", "_").lower()}.png'
    )
    plt.figure(figsize=(8, 5))
    sns.histplot(gdf_osm[column].dropna(), kde=True, bins=20)
    plt.title(f'Histogram of {column} of {study_area_name_only}', fontsize=16)
    plt.xlabel(f'Grid cells values of {column}', fontsize=16)
    plt.ylabel('Frequency', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

    with output:
        print(f"Histogram saved at: {save_path}")

# --- Bind Buttons ---
plot_button.on_click(plot_histogram)
save_button.on_click(save_histogram)

# --- Display ---
display(widgets.VBox([dropdown, plot_button, save_button, output]))

# Generate the initial plot
plot_histogram()

##### 1.2 BoxPlot Analysis

In [ ]:
# Preliminary analysis with boxplots

# Setup directory for saving boxplots
boxplot_output_dir = 'results/1_descriptive_statistics/2_boxplots/'
os.makedirs(boxplot_output_dir, exist_ok=True)

# Extract study area name function (based on selected_file)
def extract_study_area_name(selected_file):
    study_area_name, _ = os.path.splitext(selected_file)
    tokens = study_area_name.replace("-", "_").split("_")
    try:
        idx_results = tokens.index('results')
        study_area_token = tokens[idx_results - 1]
    except ValueError:
        study_area_token = tokens[-1]
    return study_area_token.replace("-", " ").replace("_", " ").title()

# Ensure that selected_file is already defined elsewhere
study_area_name_only = extract_study_area_name(selected_file)

# --- Widgets ---
dropdown_boxplot = widgets.Dropdown(
    options=[col for col in gdf_osm.columns if col != 'geometry'],
    value='leisure_name_ratio',
    description='Column:',
    disabled=False
)

plot_boxplot_button = widgets.Button(
    description='Generate Boxplot',
    button_style='success'
)

save_boxplot_button = widgets.Button(
    description='Save Boxplot as PNG',
    button_style='info',
    disabled=True
)

output_boxplot = widgets.Output()

# --- Functions ---
def plot_boxplot(b=None):
    clear_output(wait=True)
    display(widgets.VBox([dropdown_boxplot, plot_boxplot_button, save_boxplot_button, output_boxplot]))

    column = dropdown_boxplot.value
    with output_boxplot:
        output_boxplot.clear_output()
        plt.figure(figsize=(6, 6))
        sns.boxplot(y=gdf_osm[column].dropna())
        plt.title(f'Boxplot of {column} of {study_area_name_only}', fontsize=14)
        plt.ylabel(f'{column} values')
        plt.grid(False)
        plt.tight_layout()
        plt.show()

        save_boxplot_button.disabled = False

def save_boxplot(b):
    column = dropdown_boxplot.value
    save_path = os.path.join(
        boxplot_output_dir,
        f'boxplot_{column.lower()}_{study_area_name_only.replace(" ", "_").lower()}.png'
    )
    plt.figure(figsize=(6, 6))
    sns.boxplot(y=gdf_osm[column].dropna())
    plt.title(f'Boxplot of {column} of {study_area_name_only}', fontsize=14)
    plt.ylabel(f'{column} values')
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

    with output_boxplot:
        print(f"Boxplot saved at: {save_path}")

# --- Bind Buttons ---
plot_boxplot_button.on_click(plot_boxplot)
save_boxplot_button.on_click(save_boxplot)

# --- Display ---
display(widgets.VBox([dropdown_boxplot, plot_boxplot_button, save_boxplot_button, output_boxplot]))

# Generate the initial plot
plot_boxplot()

#### 2. Ordinary Least Squares (OLS) regression

In [ ]:
# Convert GeoDataFrame to DataFrame by removing the geometry column
gdf_osm_copy = gdf_osm.copy()
df_osm = gdf_osm_copy.drop(columns=['geometry'])

In [ ]:
# Check the five first and last records of DataFrame
display(df_osm)

In [ ]:
# Check the data types
df_osm.dtypes

In [ ]:
# Perform Multiple Linear Regression (OLS)
# Dynamic selection of the dependent variable and the independent variables
# Dependent variables
dependent_var_widget = Dropdown(options=[col for col in df_osm.columns if col != 'id'],
                                description='Dependent Var:',value = 'leisure_name_ratio')

# Independent variables
independent_vars_widget = SelectMultiple(
    options=[col for col in df_osm.columns if col != 'id'],
    description='Independent Vars (use ctrl to select multiple variables):',
    layout={'width': '90%', 'height': '200px'})

# Global Dictionary to store all regression summaries
regression_summaries = {}

# Global dictionary for storing R² values
r2_values = {}

# --- Utility Function ---
def extract_study_area_name(selected_file):
    study_area_name, _ = os.path.splitext(selected_file)
    tokens = study_area_name.replace("-", "_").split("_")
    try:
        idx_results = tokens.index('results')
        study_area_token = tokens[idx_results - 1]
    except ValueError:
        study_area_token = tokens[-1]
    return study_area_token.replace("-", " ").replace("_", " ").title()

# Function to run the regression
def run_regression(button):
    global regression_summaries, r2_values
    dependent_var = dependent_var_widget.value
    independent_vars = list(independent_vars_widget.value)
    X = df_osm[independent_vars]
    y = df_osm[dependent_var]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    # Print the summary for the current run
    display(model.summary())

    # To save the summary as text file
    regression_summary = model.summary().as_text()

    # Store the summary using the model name as the key
    model_name = f"{dependent_var} ~ {' + '.join(independent_vars)}"
    regression_summaries[model_name] = regression_summary

    # Store R² Adjusted values
    r2_values[model_name] = model.rsquared_adj

# Function to clear the outputs
def clear_outputs(button):
    clear_output()
    display_widgets()

# Function to save all regression results
def save_regression_results(button):
    study_area_name_only = extract_study_area_name(selected_file)
    detected_tag = dependent_var_widget.value.split('_')[0] if '_' in dependent_var_widget.value else 'unknown'
    save_filename = f'regression_summaries_{detected_tag}_{study_area_name_only.replace(" ", "_")}.txt'
    save_path = os.path.join('results/2_OLS_regression/', save_filename)

    if regression_summaries:
        with open(save_path, "w") as file:
            for model, summary in regression_summaries.items():
                file.write(f"Model: {model}\n")
                file.write(f"{summary}\n\n")
        print(f"All results saved successfully at: {save_path}")
    else:
        print("No results to save.")

# Function to Generate the Bar Chart for Adjusted R² Values
def plot_r2_bar_chart():
    # Convert the dictionary of R² values into a DataFrame for visualization
    df_r2 = pd.DataFrame(list(r2_values.items()), columns=['Model', 'Adjusted R2'])

    # Sort the models by 'Adjusted R2' values for better visualization
    df_r2_sorted = df_r2.sort_values(by='Adjusted R2', ascending=True)

    # Create the bar chart with bars closer together
    fig, ax = plt.subplots(figsize=(10, 8))
    bars = sns.barplot(x='Adjusted R2', y='Model', data=df_r2_sorted, palette='viridis', dodge=False)

    # Add labels and title for the chart
    ax.set_xlabel('Adjusted R²')
    ax.set_ylabel('Model')
    ax.set_title('Adjusted R² Values for Different Regression Models')

    # Add the values on the bars
    for bar in bars.patches:
        ax.text(
            bar.get_width(),  # get the horizontal end point of the bar
            bar.get_y() + bar.get_height() / 2,  # get the y position of the bar
            f'{bar.get_width():.2f}',  # the value we want to add as text
            va='center',  # vertical alignment
            ha='left',  # horizontal alignment
            color='black',  # text color
            fontweight='light'  # font weight
        )

    # Display the chart
    plt.tight_layout()
    plt.show()

# Function to Buttons actions
def button_actions(b):
    if b.description == 'Run Regression':
        run_regression(b)
    elif b.description == 'Clear Outputs':
        clear_outputs(b)
    elif b.description == 'Save Results':
        save_regression_results(b)
    elif b.description == 'Generate R² bar chart':
        plot_r2_bar_chart()

# Create Buttons to run the analysis
run_button = Button(description='Run Regression',
                    button_style='success')
heatmap_button = Button(description='Generate R² bar chart',
                        button_style='success')
save_button = Button(description="Save Results",
                     button_style='info')
clear_button = Button(description="Clear Outputs",
                      button_style='danger')

# Assign the button_actions function as the on_click event handler for all buttons
for button in [run_button, heatmap_button, save_button, clear_button]:
    button.on_click(button_actions)

# Organize the widgets
variable_selection_box = VBox([dependent_var_widget, independent_vars_widget])
buttons_box = VBox([run_button, heatmap_button, save_button, clear_button])

# Function to display the widgets
def display_widgets():
    hbox = HBox([variable_selection_box, buttons_box])
    display(hbox)

# Display the Widgets and the Buttons
display_widgets()

---
### Spatial Patterns Analysis

#### 3. Choropleth maps

In [ ]:
# Choropleth maps with Manual Styling Option

# Ensure that that gdf_osm and selected_file are already loaded

# Set CRS if needed
if gdf_osm.crs is None:
    gdf_osm = gdf_osm.set_crs("EPSG:4674")

# Output directory for choropleth maps
output_path = 'results/3_choropleth_maps/'

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Calculate the centroid of the union of all geometries
def calculate_centroid_of_union(gdf):
    union_poly = gdf.geometry.unary_union
    if isinstance(union_poly, MultiPolygon):
        union_poly = MultiPolygon(union_poly).convex_hull
    return [union_poly.centroid.y, union_poly.centroid.x]

# Generate choropleth map (manual style or folium built-in)
def add_choropleth(map_obj, column, method, palette, manual_style=False):
    unique_values = len(gdf_osm[column].unique())
    k = min(4, unique_values)

    if method == 'Quantiles':
        classifier = mapclassify.Quantiles(gdf_osm[column], k=k)
        class_labels = classifier.yb
        bins = [gdf_osm[column].min()] + list(classifier.bins)
        labels = [f"Q{i+1}" for i in range(len(bins)-1)]

    elif method == 'EqualInterval':
        classifier = mapclassify.EqualInterval(gdf_osm[column], k=k)
        class_labels = classifier.yb
        bins = [gdf_osm[column].min()] + list(classifier.bins)
        labels = ['' for _ in range(len(bins)-1)]

    elif method == 'NaturalBreaks':
        classifier = mapclassify.NaturalBreaks(gdf_osm[column], k=k)
        class_labels = classifier.yb
        bins = [gdf_osm[column].min()] + list(classifier.bins)
        labels = ['' for _ in range(len(bins)-1)]

    elif method == 'EqualIntervalCustom':
        bins = [0, 25, 50, 75, 100]
        class_labels = pd.cut(
            gdf_osm[column],
            bins=bins,
            labels=False,
            include_lowest=True,
            right=True
        )
        labels = ['' for _ in range(len(bins)-1)]

    else:
        raise ValueError("Classification method not supported")

    gdf_osm['class'] = class_labels

    cmap = matplotlib.colormaps.get_cmap(palette)
    colors = [matplotlib.colors.to_hex(cmap(i / max(1, len(bins)-2))) for i in range(len(bins)-1)]

    if manual_style:
        # Manual styling with GeoJson
        def style_function(feature):
            class_value = feature['properties']['class']
            if class_value is None or np.isnan(class_value):
                return {'fillOpacity': 0, 'weight': 0}
            else:
                return {
                    'fillColor': colors[int(class_value)],
                    'color': 'black',
                    'weight': 0.4,
                    'fillOpacity': 0.6
                }

        folium.GeoJson(
            data=gdf_osm.to_json(),
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(fields=['id', column])
        ).add_to(map_obj)

    else:
        # Default folium choropleth
        choropleth = folium.Choropleth(
            geo_data=gdf_osm.to_json(),
            data=gdf_osm,
            columns=['id', 'class'],
            key_on='feature.properties.id',
            fill_color=palette,
            fill_opacity=0.5,
            line_opacity=0.4,
            legend_name=''
        ).add_to(map_obj)

        folium.GeoJsonTooltip(fields=['id', column]).add_to(choropleth.geojson)

    return bins, colors, labels

# Create custom HTML legend
def create_legend_html(bins, colors, labels, method, column):
    legend_title = f"{method} classification of {column}"
    legend_html = f'''
    <div style="position: fixed;
    bottom: 50px; left: 50px; width: 300px; height: auto;
    border:2px solid grey; z-index:9999; font-size:14px; padding: 10px; background: rgba(255, 255, 255, 0.5);">
    &nbsp; {legend_title} <br>
    '''

    for i in range(len(colors)):
        color = colors[i]
        if method == 'Quantiles' and i < len(labels):
            interval_text = f"{labels[i]} ({bins[i]:.0f}-{bins[i+1]:.0f})"
        else:
            interval_text = f"{bins[i]:.0f} - {bins[i+1]:.0f}"

        legend_html += f'''
        &nbsp; <i style="background: {color}; width: 20px; height: 20px; float: left; margin-right: 4px;"></i>
        <span>{interval_text}</span><br clear="all">
        '''

    legend_html += '</div>'
    return legend_html

# Update and render the folium map
def update_map(column, method, palette, manual_style=False):
    fig = Figure(width=1080, height=650)
    centroid_coords = calculate_centroid_of_union(gdf_osm)

    m = folium.Map(location=centroid_coords, zoom_start=14, control_scale=True)
    m.add_to(fig)

    folium.TileLayer(
        tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png',
        attr='CartoDB',
        name='CartoDB light'
    ).add_to(m)

    bins, colors, labels = add_choropleth(m, column, method, palette, manual_style)

    legend_html = create_legend_html(bins, colors, labels, method, column)
    m.get_root().html.add_child(Element(legend_html))

    study_area_name, _ = os.path.splitext(selected_file)
    tokens = study_area_name.replace("-", "_").split("_")
    try:
        idx_results = tokens.index('results')
        study_area_token = tokens[idx_results - 1]
    except ValueError:
        study_area_token = tokens[-1]

    study_area_name_only = study_area_token.replace("-", " ").replace("_", " ").title()

    title_html = f'''
    <div style="position: fixed;
    top: 10px; left: 50%; transform: translate(-50%, 0); width: auto;
    border:2px solid grey; z-index:9999; font-size:16px; font-weight: bold;
    background: rgba(255, 255, 255, 0.8); text-align: center; padding: 5px;">
    Choropleth map of "{column}" of {study_area_name_only}</div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))

    display(m)

    filename = f"{column}-{study_area_name_only}-coropleth-{method}.html"
    save_path = os.path.join(output_path, filename)

    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    m.save(save_path)
    print(f"Choropleth map saved in: {save_path}")

# Widgets
column_selector = widgets.Dropdown(
    options=gdf_osm.columns,
    value='leisure_name_ratio',
    description='Column:',
    layout=Layout(width='30%', height='30px')
)

method_selector = widgets.Dropdown(
    options=['Quantiles', 'EqualInterval', 'EqualIntervalCustom', 'NaturalBreaks'],
    value='Quantiles',
    description='Method:',
    layout=Layout(width='30%', height='30px')
)

palette_selector = widgets.Dropdown(
    options=plt.colormaps(),
    value='YlGn',
    description='Palette:',
    layout=Layout(width='30%', height='30px')
)

manual_style_selector = widgets.Checkbox(
    value=False,
    description='Use Manual Style?'
)

generate_button = widgets.Button(description="Generate Map")
clear_button = widgets.Button(description="Clear Output")

def on_generate_button_clicked(b):
    clear_output(wait=True)
    display_widgets()
    update_map(column_selector.value, method_selector.value, palette_selector.value, manual_style_selector.value)

def on_clear_button_clicked(b):
    clear_output(wait=True)
    display_widgets()

def display_widgets():
    display(VBox([
        column_selector,
        method_selector,
        palette_selector,
        manual_style_selector,
        HBox([generate_button, clear_button])
    ]))

generate_button.on_click(on_generate_button_clicked)
clear_button.on_click(on_clear_button_clicked)

display_widgets()

In [ ]:
# Checking the ne column used to calculate the quantiles
print(gdf_osm['leisure_name_ratio'].describe())

#### 4. Global Moran's Index

- The bellow ceel code allows the user to select multiple variables from a GeoDataFrame and then analyze each one for spatial autocorrelation using Moran's I. The results, including the value of Moran's I and its p-value, are printed and visualized in a scatterplot for each selected variable. The selection widget's width is adjusted for better display.


- Statistical test

  - Null hypothesis represents the spatial independence of the data (Spatial distribution of the variable in question is random )
  - Confidence margin of 95% (p value < .05)

In [ ]:
# FutureWarning ignore
warnings.simplefilter(action='ignore', category=FutureWarning)

# Function to perform Moran's I analysis
def analyze_morans_i(selected_vars):
    w = weights.Queen.from_dataframe(gdf_osm, use_index=True)
    w.transform = 'r'

    for var in selected_vars:
        moran = Moran(gdf_osm[var], w)
        print(f"Moran's Index for {var}: {moran.I:.4f}, p-value: {moran.p_sim}")

        # Plot the Moran scatterplot
        plot_moran(moran, zstandard=True, figsize=(10, 4))
        plt.show()

# Button to perform the analysis and clear output
analyze_button = widgets.Button(description="Analyze Moran's I")
clear_output_button = widgets.Button(description="Clear Output")

# Create the selection widget for selecting multiple variables
select_variables = widgets.SelectMultiple(
    options=gdf_osm.columns,
    value=['leisure_name_ratio'],  # default value
    description='Variables:',
    disabled=False,
    layout={'width': '50%', 'height': '200px'}
)

# Function to handle button click event for analyzing Moran's I
def on_analyze_button_clicked(b):
    clear_output(wait=True)
    display_widgets()  # Redisplay widgets to maintain UI state
    analyze_morans_i(select_variables.value)

# Function to handle button click event for clearing the output
def on_clear_output_button_clicked(b):
    clear_output(wait=True)
    display_widgets()

# Function to display widgets with the appropriate layout
def display_widgets():
    display(VBox([
        select_variables,
        HBox([analyze_button, clear_output_button])
    ]))

# Connect the buttons to their respective event handlers
analyze_button.on_click(on_analyze_button_clicked)
clear_output_button.on_click(on_clear_output_button_clicked)

# Display the widgets initially
display_widgets()

#### 5. Hot Spot analysis using the Getis-Ord Gi statistic

In [ ]:
# Hot Spot Analysis using Getis-Ord Gi* Statistic

# Ensure that gdf_osm and selected_file are loaded

# Set CRS if missing
if gdf_osm.crs is None:
    gdf_osm = gdf_osm.set_crs("EPSG:4674")

# Output directory
output_path = 'results/5_getis_ord_maps/'

warnings.simplefilter(action='ignore', category=FutureWarning)

# Calculate centroid of union of geometries
def calculate_centroid_of_union(gdf):
    union_poly = gdf.geometry.unary_union
    if isinstance(union_poly, MultiPolygon):
        union_poly = MultiPolygon(union_poly).convex_hull
    return [union_poly.centroid.y, union_poly.centroid.x]

# Color assignment based on Z-score
def color_producer(val):
    if val > 2.0:
        return 'red' # Vermelho
    elif 1.0 < val <= 2.0:
        return 'orange' # laranja
    elif -1.0 <= val <= 1.0:
        return 'darkgray' # cinza escuro
    elif -2.0 <= val < -1.0:
        return 'darkturquoise' # Ciano
    else:
        return 'blue' # Azul

# Calculate Getis-Ord Gi*
def calculate_getis_ord_gi(gdf, var_name):
    w = weights.distance.KNN.from_dataframe(gdf, k=8)
    w.transform = 'B'
    gi = G_Local(gdf[var_name], w, star=True)
    gdf[f"Gi_Z_{var_name}"] = gi.Zs
    return gdf

# Title HTML
def create_title_html(selected_var, study_area_name_only):
    title_html = f'''
    <div style="position: fixed;
    top: 10px; left: 50%; transform: translate(-50%, 0); width: auto;
    border:2px solid grey; z-index:9999; font-size:16px; font-weight: bold;
    background: rgba(255, 255, 255, 0.8); text-align: center; padding: 5px;">
    Getis-Ord Gi* Analysis for {selected_var} of {study_area_name_only}</div>
    '''
    return title_html

# Legend HTML
def create_legend_html():
    legend_html = '''
    <div style="position: fixed;
    bottom: 50px; left: 50px; width: 270px; height: auto;
    border:2px solid grey; z-index:9999; font-size:14px;
    background: rgba(255, 255, 255, 0.8);">
    &nbsp; <b>Legend</b> <br>
    &nbsp; Significant hot-spot (Z > 2.0) &nbsp; <i style="background:red;width:10px;height:10px;display:inline-block;"></i><br>
    &nbsp; Moderate hot-spot (1.0 < Z ≤ 2.0) &nbsp; <i style="background:orange;width:10px;height:10px;display:inline-block;"></i><br>
    &nbsp; Non-significant (-1.0 ≤ Z ≤ 1.0) &nbsp; <i style="background:darkgray;width:10px;height:10px;display:inline-block;"></i><br>
    &nbsp; Moderate cold-spot (-2.0 ≤ Z < -1.0) &nbsp; <i style="background:darkturquoise;width:10px;height:10px;display:inline-block;"></i><br>
    &nbsp; Significant cold-spot (Z < -2.0) &nbsp; <i style="background:blue;width:10px;height:10px;display:inline-block;"></i>
    </div>
    '''
    return legend_html

# Main function to analyze and generate map
def analyze_getis_ord_gi(selected_var):
    display("Wait: processing Getis-Ord Gi* analysis...")
    gdf_with_gi = calculate_getis_ord_gi(gdf_osm.copy(), selected_var)
    centroid_coords = calculate_centroid_of_union(gdf_with_gi)
    fig = Figure(width=1080, height=650)

    m = folium.Map(location=centroid_coords, zoom_start=14, control_scale=True)
    fig.add_child(m)

    folium.TileLayer(
        tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png',
        attr='CartoDB',
        name='CartoDB light'
    ).add_to(m)

    for _, row in gdf_with_gi.iterrows():
        centroid = row.geometry.centroid if isinstance(row.geometry, MultiPolygon) else row.geometry
        z_value = row[f"Gi_Z_{selected_var}"]
        folium.CircleMarker(
            location=[centroid.y, centroid.x],
            radius=3,
            color=color_producer(z_value),
            fill=True,
            fill_opacity=1
        ).add_to(m)

    def style_function(feature):
        value = feature['properties'][f"Gi_Z_{selected_var}"]
        return {
            'fillColor': color_producer(value),
            'color': 'gray',
            'weight': 1,
            'fillOpacity': 0.2,
            'lineOpacity': 0.7
        }

    folium.GeoJson(
        gdf_with_gi,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=[f"Gi_Z_{selected_var}"],
            aliases=["Z-score:"],
            localize=True
        )
    ).add_to(m)

    study_area_name, _ = os.path.splitext(selected_file)
    tokens = study_area_name.replace("-", "_").split("_")
    try:
        idx_results = tokens.index('results')
        study_area_token = tokens[idx_results - 1]
    except ValueError:
        study_area_token = tokens[-1]

    study_area_name_only = study_area_token.replace("-", " ").replace("_", " ").title()

    m.get_root().html.add_child(Element(create_legend_html()))
    m.get_root().html.add_child(Element(create_title_html(selected_var, study_area_name_only)))

    display(fig)

    output_filename = f"{selected_var}-{study_area_name_only}-getisord.html"
    save_path = os.path.join(output_path, output_filename)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    m.save(save_path)
    display(f"Map saved at: {save_path}")

# Widgets
select_variable_widget = widgets.Select(
    options=gdf_osm.columns,
    value='leisure_name_ratio', # default var
    description='Variable:',
    layout={'width': '30%', 'height': '150px'}
)

analyze_button = widgets.Button(description="Analyze and Show Map",
                                button_style='success')
clear_output_button = widgets.Button(description="Clear Output",
                                     button_style='danger')

box_layout = VBox([select_variable_widget, HBox([analyze_button, clear_output_button])])

def on_analyze_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)
    analyze_getis_ord_gi(select_variable_widget.value)

def on_clear_output_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)

analyze_button.on_click(on_analyze_button_clicked)
clear_output_button.on_click(on_clear_output_button_clicked)

display(box_layout)

'Wait: processing Getis-Ord Gi* analysis...'

'Map saved at: results/5_getis_ord_maps/amenity_name_ratio-Passare-getisord.html'

#### 6. Geographically Weighted Regression (GWR)

##### Reproject the GeoDataFrame to apply GWR

In [ ]:
# Function to calculate the centroid of a GeoDataFrame

def get_utm_zone_crs(gdf):
    bounds = gdf.total_bounds
    bbox = box(bounds[0], bounds[1], bounds[2], bounds[3])
    centroid = bbox.centroid
    
    # Determine the UTM zone from the centroid's longitude
    utm_zone = int((centroid.x + 180) / 6) + 1
    hemisphere = 'north' if centroid.y > 0 else 'south'

    # Create a UTM CRS based on the centroid location
    utm_crs = pyproj.CRS(f"+proj=utm +zone={utm_zone} +{hemisphere} +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    
    return utm_crs

# Set the initial CRS if necessary
gdf_osm = gdf_osm.set_crs("EPSG:4674")

# Obtain the appropriate UTM CRS
utm_crs = get_utm_zone_crs(gdf_osm)

# Project the GeoDataFrame to the detected UTM CRS
gdf_osm_projected = gdf_osm.to_crs(utm_crs.to_string())

gdf_osm_projected.crs

##### Applying VIF (Variance Inflation Factor) to identify multicollinearity

In [ ]:
# Applying VIF (Variance Inflation Factor) to identify multicollinearity
# among independent variables in regression models

# --- Setup ---

# Output directories
vif_output_dir = 'results/6_gwr/vif/'
corr_output_dir = 'results/6_gwr/vif/correlation_matrix/'

# Ensure directories exist
os.makedirs(vif_output_dir, exist_ok=True)
os.makedirs(corr_output_dir, exist_ok=True)

warnings.simplefilter(action='ignore', category=FutureWarning)

# Function to extract study area name from selected_file
def extract_study_area_name(selected_file):
    study_area_name, _ = os.path.splitext(selected_file)
    tokens = study_area_name.replace("-", "_").split("_")
    try:
        idx_results = tokens.index('results')
        study_area_token = tokens[idx_results - 1]
    except ValueError:
        study_area_token = tokens[-1]
    return study_area_token.replace("-", " ").replace("_", " ").title()

# --- Widgets ---
select_x_variables_widget = SelectMultiple(
    options=[col for col in gdf_osm_projected.columns if col != 'geometry'],
    description='Independent Variables:',
    disabled=False,
    layout=Layout(width='40%', height='200px')
)

vif_button = Button(description="Calculate VIF")
correlation_button = Button(description="Show Correlation Matrix")
clear_button = Button(description="Clear Results")

box_layout = VBox([
    select_x_variables_widget,
    HBox([vif_button, correlation_button, clear_button])
])

display(box_layout)

# Global variable to store detected OSM tag
detected_tag = 'unknown'

# --- Function to calculate VIF, save CSV, and show download link ---
def calculate_vif(b):
    global detected_tag # Declare usage of the detected_tag as global variable
    clear_output(wait=True)
    display(box_layout)
    selected_vars = list(select_x_variables_widget.value)

    if len(selected_vars) < 2:
        print("Please select at least two variables to calculate VIF.")
        return

    variables = gdf_osm_projected[selected_vars]

    vif_data = pd.DataFrame({
        "feature": variables.columns,
        "VIF": [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]
    })

    display(vif_data)

    # Detect OSM Tag dynamically
    tags_in_selection = [col.split('_')[0] for col in selected_vars if '_' in col]
    if tags_in_selection:
        detected_tag = tags_in_selection[0]
    else:
        detected_tag = 'unknown'

    # Add study area name
    study_area_name_only = extract_study_area_name(selected_file)

    # Save VIF to CSV
    save_path = os.path.join(vif_output_dir, f'vif_{detected_tag}_{study_area_name_only}.csv')
    vif_data.to_csv(save_path, index=False)
    print(f"\u2705 VIF saved at: {save_path}")

    # Provide download link
    display(FileLink(save_path, result_html_prefix="Click to download VIF file: "))

# --- Function to show correlation matrix, save PNG, and show download link ---
def show_correlation_matrix(b):
    clear_output(wait=True)
    display(box_layout)
    selected_vars = list(select_x_variables_widget.value)

    if len(selected_vars) < 2:
        print("Please select at least two variables to view the correlation matrix.")
        return

    variables = gdf_osm_projected[selected_vars]
    corr_matrix = variables.corr()

    # Plot and save
    plt.figure(figsize=(8,6))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlation Matrix of Independet Variables')

    # Add study area name
    study_area_name_only = extract_study_area_name(selected_file)

    corr_save_path = os.path.join(corr_output_dir, f'correlation_matrix_{detected_tag}_{study_area_name_only}.png')
    plt.savefig(corr_save_path, dpi=300, bbox_inches='tight')
    plt.show()

    print(f"\u2705 Correlation matrix saved at: {corr_save_path}")
    display(FileLink(corr_save_path, result_html_prefix="Click to download correlation matrix: "))

# --- Function to clear the output ---
def on_clear_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)

# --- Event bindings ---
vif_button.on_click(calculate_vif)
correlation_button.on_click(show_correlation_matrix)
clear_button.on_click(on_clear_button_clicked)

##### Check explained variance ratio

In [ ]:
# Checking the explained variance

features = [col for col in gdf_osm_projected.columns if col != 'geometry' and col not in ['leisure_name_ratio', 'building_name_ratio', 'amenity_name_ratio']]
select_features_widget = SelectMultiple(
    options=features,
    value=[features[0]],
    description='Variables:',
    disabled=False,
    layout=Layout(width='50%', height='120px')
)

plot_button = Button(description="Plot Explained Variance")
clear_button = Button(description="Clear Results")

# Layout setup
box_layout = VBox([select_features_widget,
                   HBox([plot_button, clear_button])
                   ])
display(box_layout)

# Function to plot explained variance
def plot_variance(b):
    clear_output(wait=True)
    display(box_layout)
    selected_features = list(select_features_widget.value)
        
    # Standardizing the selected features
    x = gdf_osm_projected[selected_features]
    x_scaled = StandardScaler().fit_transform(x)
        
     # Applying PCA and capturing the explained variance
    pca = PCA()
    pca.fit(x_scaled)
    explained_variance = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance)
        
    # Plotting
    plt.figure(figsize=(10, 5))
    plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.5, align='center', label='Individual explained variance')
    plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='Cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal component index')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

# Function to clear results
def clear_results(b):
    clear_output(wait=True)
    display(box_layout)

# Binding the buttons to their respective functions
plot_button.on_click(plot_variance)
clear_button.on_click(clear_results)

##### Perform Geographically Weighted Regression (GWR)

In [ ]:
# Perform Geographically Weighted Regression (GWR) with PCA

gdf_osm_projected = gdf_osm_projected.copy()

# --- Setup Directories ---

gwr_output_dir = 'results/6_gwr/gwr_pca/'

os.makedirs(gwr_output_dir, exist_ok=True)

warnings.simplefilter(action='ignore', category=FutureWarning)

# --- Utility Functions ---

def extract_study_area_name(selected_file):
    study_area_name, _ = os.path.splitext(selected_file)
    tokens = study_area_name.replace("-", "_").split("_")
    try:
        idx_results = tokens.index('results')
        study_area_token = tokens[idx_results - 1]
    except ValueError:
        study_area_token = tokens[-1]
    return study_area_token.replace("-", " ").replace("_", " ").title()

# --- Widgets ---
select_y_variable_widget = Dropdown(
    options=[col for col in gdf_osm_projected.columns if col != 'geometry'],
    value='leisure_name_ratio',  # Default suggested
    description='Dependent Variable:',
    disabled=False
)

select_x_variables_widget = SelectMultiple(
    options=[col for col in gdf_osm_projected.columns if col != 'geometry'],
    description='Independent Variables:',
    disabled=False,
    layout=Layout(width='50%', min_width='300px', height='200px')
)

analyze_button = Button(description="Perform GWR with PCA", layout=Layout(width='auto', min_width='200px'))
clear_button = Button(description="Clear Results", layout=Layout(width='auto', min_width='120px'))

box_layout = VBox([
    select_y_variable_widget,
    select_x_variables_widget,
    HBox([analyze_button, clear_button], layout=Layout(margin='10px 0'))
], layout=Layout(margin='0 0 10px 0'))

display(box_layout)

# --- Main Analysis Function ---
def on_analyze_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)
    display(" Processing GWR with PCA, please wait...")

    selected_y_var = select_y_variable_widget.value
    selected_x_vars = list(select_x_variables_widget.value)

    if len(selected_x_vars) < 2:
        display("Please select at least two independent variables.")
        return

    # Detect TAG from independent variables
    tags_in_selection = [col.split('_')[0] for col in selected_x_vars if '_' in col]
    detected_tag = tags_in_selection[0] if tags_in_selection else 'unknown'

    study_area_name_only = extract_study_area_name(selected_file)

    # --- Standardize Features ---
    features = gdf_osm_projected[selected_x_vars]
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    # --- Perform PCA ---
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(features_scaled)
    principal_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
    principal_df['DependentVar'] = gdf_osm_projected[selected_y_var]

    # --- Save PCA Loadings ---
    pca_loadings = pd.DataFrame(
        pca.components_,
        columns=selected_x_vars,
        index=['PC1', 'PC2']
    )
    display(pca_loadings)

    loadings_save_path = os.path.join(
        gwr_output_dir, f'pca_loadings_{detected_tag}_{study_area_name_only}.csv'
    )
    pca_loadings.to_csv(loadings_save_path)
    print(f"PCA loadings saved at: {loadings_save_path}")

    # --- Prepare coordinates and target ---
    coords = np.array(list(zip(gdf_osm_projected.geometry.centroid.x, gdf_osm_projected.geometry.centroid.y)))
    y = principal_df['DependentVar'].values.reshape((-1, 1))
    X = principal_df[['PC1', 'PC2']].values

    # --- Perform GWR ---
    gwr_selector = Sel_BW(coords, y, X, kernel='bisquare', fixed=False)
    bw = gwr_selector.search()
    gwr_model = GWR(coords, y, X, bw).fit()

    # --- Store GWR results with TAG in column names ---
    gdf_osm_projected[f'Local_R2_{detected_tag}'] = gwr_model.localR2
    gdf_osm_projected[f'PC1_coef_{detected_tag}'] = gwr_model.params[:, 1]
    gdf_osm_projected[f'PC2_coef_{detected_tag}'] = gwr_model.params[:, 2]

    # --- Plot Results ---
    fig, axs = plt.subplots(1, 3, figsize=(18, 5))

    scatter1 = axs[0].scatter(coords[:, 0], coords[:, 1], c=gdf_osm_projected[f'Local_R2_{detected_tag}'], cmap='viridis_r') #YlGn
    axs[0].set_title(f'Local R²')
    #axs[0].set_title(f'Local R² \n{selected_y_var} of {study_area_name_only}')
    plt.colorbar(scatter1, ax=axs[0])

    scatter2 = axs[1].scatter(coords[:, 0], coords[:, 1], c=gdf_osm_projected[f'PC1_coef_{detected_tag}'], cmap='coolwarm')
    axs[1].set_title(f'GWR Local Regression Coefficient - PC1')
    #axs[1].set_title(f'GWR Local Regression Coefficient - PC1\n{selected_y_var} of {study_area_name_only}')
    plt.colorbar(scatter2, ax=axs[1])

    scatter3 = axs[2].scatter(coords[:, 0], coords[:, 1], c=gdf_osm_projected[f'PC2_coef_{detected_tag}'], cmap='coolwarm')
    axs[2].set_title(f'GWR Local Regression Coefficient - PC2')
    #axs[2].set_title(f'GWR Local Regression Coefficient - PC2\n{selected_y_var} of {study_area_name_only}')
    plt.colorbar(scatter3, ax=axs[2])

    plt.tight_layout()

    map_save_path = os.path.join(
        gwr_output_dir, f'gwr_pca_maps_{detected_tag}_{study_area_name_only}.png'
    )
    plt.savefig(map_save_path, dpi=300, bbox_inches='tight')
    plt.show()

    print(f"GWR maps saved at: {map_save_path}")

# --- Clear Function ---
def on_clear_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)

# --- Bind Buttons ---
analyze_button.on_click(on_analyze_button_clicked)
clear_button.on_click(on_clear_button_clicked)

In [ ]:
display(gdf_osm_projected)

In [ ]:
# Save the final GeoDataFrame as GeoJSON
gdf_osm_projected.to_file(os.path.join(gwr_output_dir, 'gdf_osm_projected_gwr_pca_results_passare.geojson'), driver='GeoJSON')
print(f"GeoJSON saved at: {gwr_output_dir}")

In [ ]:
# Perform GWR + PCA Visualization (without recalculating anything!)

# --- Setup ---
gwr_output_dir = 'results/6_gwr/gwr_pca/folium_maps/'
os.makedirs(gwr_output_dir, exist_ok=True)
warnings.simplefilter(action='ignore', category=FutureWarning)

# --- Utility Function ---
def extract_study_area_name(selected_file):
    study_area_name, _ = os.path.splitext(selected_file)
    tokens = study_area_name.replace("-", "_").split("_")
    try:
        idx_results = tokens.index('results')
        study_area_token = tokens[idx_results - 1]
    except ValueError:
        study_area_token = tokens[-1]
    return study_area_token.replace("-", " ").replace("_", " ").title()

# --- Widgets ---
map_selector = Dropdown(
    options=[],
    description='Select Map:',
    disabled=True
)

prepare_button = Button(description="Press here first to prepare maps", layout=Layout(width='auto', min_width='200px'))
run_button = Button(description="Display Selected GWR Map on Folium", layout=Layout(width='auto', min_width='200px'), disabled=True)
clear_button = Button(description="Clear Output", layout=Layout(width='auto', min_width='120px'))

output = Output()

box_layout = VBox([
    HBox([prepare_button, clear_button], layout=Layout(margin='10px 0')),
    map_selector,
    run_button,
    output
], layout=Layout(margin='0 0 10px 0'))

display(box_layout)

# --- Global ---
saved_maps = {}

# --- Main Function ---
def on_prepare_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)
    output.clear_output()
    with output:
        print("Preparing maps from existing GWR + PCA results...")

    study_area_name_only = extract_study_area_name(selected_file)

    gdf_to_map = gdf_osm_projected.to_crs(epsg=4326)

    # Detect GWR result columns automatically
    variables = {}
    for col in gdf_to_map.columns:
        if col.startswith(('Local_R2_', 'PC1_coef_', 'PC2_coef_')):
            # Extract Tag
            tag = col.split('_')[-1]  # leisure, building, amenity...
            if 'Local_R2' in col:
                title = f'GWR Local R² for {tag}'
            elif 'PC1_coef' in col:
                title = f'GWR Local Regression Coefficient - PC1 for {tag}'
            elif 'PC2_coef' in col:
                title = f'GWR Local Regression Coefficient - PC2 for {tag}'
            else:
                title = col
            variables[col] = title

    if not variables:
        with output:
            print("No GWR or PCA results found in the data!")
        return

    saved_maps.clear()

    for var_col, var_title in variables.items():
        values = gdf_to_map[var_col]
        norm = plt.Normalize(vmin=values.min(), vmax=values.max())
        cmap = plt.get_cmap('coolwarm')

        m = folium.Map(
            location=[gdf_to_map.geometry.centroid.y.mean(), gdf_to_map.geometry.centroid.x.mean()],
            zoom_start=14,
            control_scale=True
        )

        # Basemap
        TileLayer(
            tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png',
            attr='CartoDB',
            name='CartoDB light'
        ).add_to(m)

        # Transparent grid
        GeoJson(
            gdf_to_map,
            style_function=lambda x: {
                'fillColor': 'white',
                'fill': True,
                'color': 'gray',
                'weight': 1,
                'fillOpacity': 0.01,
                'lineOpacity': 0.6
            },
            tooltip=GeoJsonTooltip(
                fields=[var_col],
                aliases=[f'{var_title}: '],
                localize=True
            )
        ).add_to(m)

        # Circle markers colored by value
        for _, row in gdf_to_map.iterrows():
            centroid = row.geometry.centroid if isinstance(row.geometry, MultiPolygon) else row.geometry
            val = row[var_col]
            color = matplotlib.colors.to_hex(cmap(norm(val)))
            CircleMarker(
                location=[centroid.y, centroid.x],
                radius=3,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=1
            ).add_to(m)
        
        # Color scale for the legend
        colormap = cm.LinearColormap(
            colors=['blue', 'white', 'red'],
            vmin=values.min(),
            vmax=values.max()
        )
        colormap.caption = var_title
        colormap.add_to(m)

        # Title
        title_html = f'''
        <div style="position: fixed; top: 10px; left: 50%; transform: translate(-50%, 0);
        width: auto; border:2px solid gray; z-index:9999; font-size:16px; font-weight: bold;
        background: rgba(255, 255, 255, 0.8); text-align: center; padding: 5px;">
        {var_title}<br>{study_area_name_only}
        </div>
        '''
        m.get_root().html.add_child(folium.Element(title_html))

        # Save map
        map_save_path = os.path.join(
            gwr_output_dir, f'gwr_pca_{var_col.lower()}_{study_area_name_only.replace(" ", "_").lower()}.html'
        )
        m.save(map_save_path)
        saved_maps[var_title] = m

    map_selector.options = list(saved_maps.keys())
    map_selector.disabled = False
    run_button.disabled = False

    with output:
        print(f"Maps ready! \nFiles saved at: {gwr_output_dir}")

# --- Display Map ---
def on_run_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)
    output.clear_output()
    selected_map_title = map_selector.value
    if selected_map_title in saved_maps:
        display(saved_maps[selected_map_title])

# --- Clear Output ---
def on_clear_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)

# --- Bind ---
prepare_button.on_click(on_prepare_button_clicked)
run_button.on_click(on_run_button_clicked)
clear_button.on_click(on_clear_button_clicked)